In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, 
                                            decoder_hidden_units = 100)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3, 
        save_path = "checkpoints/seq2seq_rnn.pt")

Net parameters: 207,195



Epoch:    1, Loss: 2.5198
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' ee  te  e', ' e  te  e ', '   ne  e e', '  ne  e e ', ' ne  e e  ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
tensor([-11.8300, -11.8100, -12.8500, -10.4100,  -9.8600], device='cuda:0')
sample
[' te aun pa', 'wiis\nchce ', "ron s Q', ", 'gin rnet h', '\n wntebrt(']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['e         ', 'e         ', '        . ', 'e         ', '        . '],
 ['        . ', '        . ', '         a', '        . ', '         a'],
 ['         a', '         a', 'e         ', '         a', 'e         '],
 ['         t', '         t', '         t', '         t', '         t']]
tensor([[-11.8300, -12.0000, -12.0400, -11.7300, -11.6100],
        [-14.1500, -14.3000, -14.3900, -13.8600, -13.9700],
        [-14.1900, -14.3500, -14.6900, -14.0900, -14.2600],
        [-14.4800, -14.


Epoch:    2, Loss: 1.9141
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' h aos  te', 'ehaote te ', ' tote te e', '  te te ee', ' le te eet']
greedy_search
[' the the t', 'e the the ', ' the the t', ' an the th', ' the the t']
tensor([-10.8600, -11.1100, -10.9500, -12.4800, -10.9500], device='cuda:0')
sample
["\nnneond'n\n", 'ard K-tala', 'chens fon ', "ark ak'h f", 'ses the th']
beam_search
[['s the the ', 'e the the ', 'the the th', ' the the t', ' the the t'],
 [' the the t', 'the the th', ' the the t', 'the the th', 'e the the '],
 ['the the th', 'es the the', ' the thes ', 'e the the ', 'the the th'],
 [' the thes ', 'the the to', ' the there', ' the thes ', ' the thes '],
 [' the the p', 'e the ther', ' the the p', ' the the p', ' the the p']]
tensor([[-10.8600, -10.7700, -11.1200, -10.9300, -10.9200],
        [-10.8700, -11.1100, -11.2600, -10.9400, -11.0100],
        [-11.0000, -11.6000, -11.9900, -10.9600, -11.0100],
        [-11.7300, -12.


Epoch:    3, Loss: 1.6354
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aos   e', 'ehoote te ', 'etote te e', 'eete te ee', ' lerte e.t']
greedy_search
[' the the t', 'en the the', 'e the the ', 'e the the ', ' andereng.']
tensor([-10.4700, -10.9100,  -9.5100, -10.3700, -16.5600], device='cuda:0')
sample
['igief s Kt', 'ke bine, a', 'upthis tou', 'ontighin g', ' oind wock']
beam_search
[['s the the ', 'e the the ', 'e the the ', 'e the the ', 'ing the th'],
 [' the the t', 'the the th', 'ing the th', 'ing the th', ' the the t'],
 [' the thes ', 'e the ther', ' the the t', ' the the t', ' the thes '],
 [' the the p', 'e the thes', 'e the ther', ' the the p', ' the the p'],
 [' the there', 'the the pa', ' the the p', ' the there', ' the there']]
tensor([[ -9.9000, -10.4100,  -9.9300, -10.1500, -10.9200],
        [-10.4900, -11.9000, -11.0000, -11.0800, -11.0000],
        [-10.5200, -12.0900, -11.2800, -11.2800, -11.0300],
        [-10.6000, -12.15


Epoch:    4, Loss: 1.5009
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e autente', 'eeoutente ', 'etuterte e', 'outerte e.', 'erertere.t']
greedy_search
[' the pann ', 'en the the', 'e the the ', 'ore the pa', 'erenting t']
tensor([-12.5200, -11.1500,  -9.7700, -13.1600, -13.5500], device='cuda:0')
sample
['n\nlale to ', 'us. The fo', 'iom at can', 'liet posen', 'amaged ToO']
beam_search
[['s the the ', 'en the the', 'ed the the', 'on the the', 'ing the th'],
 ['s the and ', 'ed the the', 'en the the', 'and the th', 'ing the pa'],
 [' the thes ', 'ing the th', 'ing the th', 'ing the th', 'ing the Kt'],
 [' the there', 'ing the pa', 'ing the pa', 'ing the pa', 'ing the an'],
 ['s the ther', 'ing the an', 'ing the an', 'ing the an', 'ing the to']]
tensor([[-10.3800, -10.8200, -10.3600, -10.4500, -10.6100],
        [-11.2000, -10.9400, -10.4500, -10.9700, -10.8100],
        [-11.2500, -11.3300, -10.9000, -10.9900, -11.0200],
        [-11.5400, -11.6


Epoch:    5, Loss: 1.4157
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ne autente', 'eeoutentyn', 'etutentyre', 'outen yre.', 'erentyre.t']
greedy_search
['ng the paw', 'ent the pa', 'e the pawn', 'on the paw', 'erengreng.']
tensor([-11.5100, -13.3400, -11.6400, -11.8800, -14.8500], device='cuda:0')
sample
['g QR and.\n', 'enn. It to', ' hamlayig,', 'a no his o', ' Kist.\n\n E']
beam_search
[['s the King', 'ed and the', 'ing the pa', 'on the the', 'ing the pa'],
 ['s the the ', 'en the the', 'ing the th', 'ing the pa', 'ing the th'],
 ['s the and ', 'ken the th', 'ing the Ki', 'ing the th', 'ing there '],
 ['ther the t', 'ken the pa', 'ing the to', 'ing the Ki', 'ing the Ki'],
 ['ther the p', 'ken the Ki', 'ing the an', 'ing the an', 'ing the to']]
tensor([[-10.2900, -10.5200, -10.6300, -10.1100, -10.3800],
        [-10.6700, -10.9900, -10.7900, -10.6400, -10.5300],
        [-11.0500, -11.5900, -10.8500, -10.8000, -10.6200],
        [-11.5300, -11

In [14]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, 
                                            decoder_hidden_units = 100)
net.load_state_dict(torch.load("checkpoints/seq2seq_rnn.pt"))
net.to(device)
test = net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)

test[1][:20]

Net parameters: 207,195


tensor([[-18.6263, -19.6530, -20.1133, -20.1255, -20.6161],
        [-20.9326, -21.5159, -21.6976, -21.8533, -22.1528],
        [-15.1424, -15.9284, -16.3301, -16.3430, -16.9841],
        [-13.6844, -13.9842, -14.1003, -14.4234, -14.7703],
        [-18.7930, -19.4146, -19.5911, -19.7269, -20.0176],
        [-18.5738, -21.1258, -21.5284, -21.5585, -21.9503],
        [-18.2225, -20.4450, -20.6791, -20.8843, -21.3122],
        [-18.1692, -19.0812, -19.5159, -19.5944, -19.6843],
        [-20.0627, -20.0858, -20.1163, -20.3464, -20.5535],
        [-17.8215, -18.5229, -18.5896, -18.9478, -19.5764],
        [-19.4423, -20.9529, -21.1818, -21.4537, -21.4704],
        [-20.4152, -20.8069, -21.4923, -21.8645, -21.8892],
        [-15.2188, -16.2112, -16.4349, -16.8548, -16.9504],
        [-16.0626, -16.5967, -17.6951, -18.2410, -18.2728],
        [-19.0831, -19.2016, -20.3759, -20.3784, -20.5128],
        [-18.3804, -21.1958, -21.3227, -21.7145, -21.7556],
        [-18.9517, -19.9929, -20.1449, -